# my-1st-try

In [ ]:
#pip list
print("はじめ")

# Vision-Trans-Former with keras(no Finetuning)

In [ ]:
import os, sys, time
from datetime import datetime
import random

import numpy as np
import pandas as pd
from glob import glob

from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
import cv2
from PIL import Image

import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 60)

In [1]:
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.vgg19 import VGG19

import tensorflow as tf
import keras as K
from keras.utils import np_utils, to_categorical
from keras.models import Sequential, load_model
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import load_img, img_to_array, save_img, ImageDataGenerator
from keras.initializers import TruncatedNormal, Constant
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation, BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import RMSprop, SGD, Adam, Nadam, Adamax, Adadelta
from keras.callbacks import EarlyStopping, ModelCheckpoint, History, LearningRateScheduler, ReduceLROnPlateau, LambdaCallback, CSVLogger
from keras import models, layers

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


from efficientnet.keras import EfficientNetB4,EfficientNetB3,EfficientNetB2,EfficientNetB1,EfficientNetB0,EfficientNetB5,EfficientNetB6,EfficientNetB7


Using TensorFlow backend.


In [ ]:
print(tf.__version__)
gpu_id = 0

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.list_physical_devices('GPU')

# 使うGPUの設定
tf.config.set_visible_devices(physical_devices[gpu_id], 'GPU')

# 動的メモリアロケート設定
tf.config.experimental.set_memory_growth(physical_devices[gpu_id], False)

In [ ]:
# config
data_seed = 94
KFold_seed = 94
EPOCHS = 10
PATCH_SIZE = 4
BATCH_SIZE = 64 #100
IMAGE_SIZE = 512 #314 #512 #64 #225
CH_NUM = 3

#学習中のtrain-data分割割合
training_percentage = 0.9

# 最適化関数
#opt = Adam(lr=0.01)
#opt = Adadelta(lr=1.0)
#opt = Adamax(lr=0.002)
#opt = Nadam(lr=0.002)
opt = "adam"                             #最適化関数は何使う？"SDG","adam","RMSprop", "nadam", "adadelta"

# 損失係数
LOSS = "categorical_crossentropy"        #損失はどうする？"categorical_crossentropy","sparse_categorical_crossentropy",""
METRICS = ["accuracy"]                   #最適化する計量は？"accuracy","",""

In [ ]:
# 画像を取得する関数(RGBで処理したい)
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    # リサイズする
    im_rgb = cv2.resize(im_rgb , (IMAGE_SIZE, IMAGE_SIZE))
    return im_rgb


In [ ]:
# GPU使えているよね？？
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

# kaggle用
# 画像のful-pathリスト作成
test_imgs = glob("../input/cassava-leaf-disease-classification/test_images/**.jpg")
# 画像のful-pathリスト作成
imgs = glob("../input/cassava-leaf-disease-classification/train_images/**.jpg")
# 画像の名称リスト
train_img_list = os.listdir(os.path.join("../input/cassava-leaf-disease-classification/train_images"))
# 画像のラベル名リスト
train_label = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
# 学習データ作成準備
samples_df = shuffle(train_label, random_state=94)
samples_df["label"] = samples_df["label"].astype("str")
samples_df.head()

# 画像のラベル詳細について
#desease_list = pd.read_json("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json")

In [ ]:
# GPGPUの使用率を確認するやつ
import subprocess
import json
import pprint

DEFAULT_ATTRIBUTES = (
    'index',
    'uuid',
    'name',
    'timestamp',
    'memory.total',
    'memory.free',
    'memory.used',
    'utilization.gpu',
    'utilization.memory'
)

def get_gpu_info(nvidia_smi_path='nvidia-smi', keys=DEFAULT_ATTRIBUTES, no_units=True):
    nu_opt = '' if not no_units else ',nounits'
    cmd = '%s --query-gpu=%s --format=csv,noheader%s' % (nvidia_smi_path, ','.join(keys), nu_opt)
    output = subprocess.check_output(cmd, shell=True)
    lines = output.decode().split('\n')
    lines = [ line.strip() for line in lines if line.strip() != '' ]

    return [ { k: v for k, v in zip(keys, line.split(', ')) } for line in lines ]

In [ ]:
pprint.pprint(get_gpu_info())

In [ ]:
# local用
# 画像のful-pathリスト作成
test_imgs = glob("/Users/kushi/TechLife/kaggle_road/06_cassava/dataset/test_images/**.jpg")
# 画像のful-pathリスト作成
imgs = glob("/Users/kushi/TechLife/kaggle_road/06_cassava/dataset/train_images/**.jpg")
# 画像の名称リスト
train_img_list = os.listdir(os.path.join("/Users/kushi/TechLife/kaggle_road/06_cassava/dataset/train_images"))
# 画像のラベル名リスト
train_label = pd.read_csv("/Users/kushi/TechLife/kaggle_road/06_cassava/dataset/train.csv")
# 学習データ作成準備
samples_df = shuffle(train_label, random_state=94)
samples_df["label"] = samples_df["label"].astype("str")
samples_df.head()

# 画像のラベル詳細について
#desease_list = pd.read_json("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json")


samples_train = samples_df
samples_val =samples_df

In [ ]:
# 学習データと評価データに分ける準備
training_item_count = int(len(samples_df)*training_percentage)
validation_item_count = len(samples_df)-int(len(samples_df)*training_percentage)
training_df = samples_df[:training_item_count]
validation_df = samples_df[training_item_count:]

In [ ]:
training_df.shape, validation_df.shape

In [ ]:
print(training_df["label"].value_counts())
print(validation_df["label"].value_counts())

# ランダムでサンプルデータから、学習セットを生成する

In [ ]:
# 学習用の準備
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
target_size = (IMAGE_SIZE, IMAGE_SIZE)
dropout_rate = 0.4
classes_to_predict = sorted(training_df.label.unique())
CLASS = len(classes_to_predict)
print("クラス数は：", CLASS)

In [ ]:
# training_dfのランダム生成関数
# fut_generatorで交差検証するのに使う

def train_data_gen(training_item_count, data_seed):
    global samples_train
    samples_train = shuffle(samples_train, random_state = data_seed)
    samples_train["label"] = samples_train["label"].astype("str")
    samples_train = samples_train[:training_item_count]
    
    return samples_train

# validation_dfのランダム生成関数
def val_data_gen(training_item_count, data_seed):
    global samples_val
    samples_val = shuffle(samples_val, random_state = data_seed)
    samples_valid = samples_val
    samples_valid["label"] = samples_valid["label"].astype("str")
    samples_valid = samples_valid[training_item_count:]
    
    return samples_valid

In [ ]:
# テストデータを確認してみる
plt.imshow(get_img(test_imgs[0]))

In [ ]:
tes_img = cv2.imread(test_imgs[0])
tes_img = cv2.resize(tes_img , (IMAGE_SIZE, IMAGE_SIZE))
tes_img.shape

In [ ]:
plt.imshow(tes_img)

In [ ]:
#submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
submission = pd.read_csv("/Users/kushi/TechLife/kaggle_road/06_cassava/dataset/sample_submission.csv")
submission.head()

# 学習画像を準備する部分

datagen=ImageDataGenerator(rescale=1./255)
train_generator=datagen.flow_from_dataframe(
    dataframe=training_df,
    directory=".\train_imgs",
    x_col="image_id",
    y_col="label",
    class_mode="sparse",
    target_size=(32,32),
    batch_size=32)

参考のやつ
ImageDataGenerator(validation_split = 0.2,
                                     preprocessing_function = None,
                                     zoom_range = 0.15,
                                     cval = 0.,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     fill_mode = 'nearest',
                                     shear_range = 0.15,
                                     height_shift_range = 0.15,
                                     width_shift_range = 0.15) \

train_datagen = ImageDataGenerator(
    rotation_range=30,
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip = True,
    height_shift_range = 0.15,
    width_shift_range = 0.15
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe = training_df,
    #dataframe = train_data_gen(training_item_count, data_seed),
    x_col='image_id',
    y_col='label',
    directory='/Users/kushi/TechLife/kaggle_road/06_cassava/dataset/train_images/',
    target_size=target_size,
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='categorical',
    seed = 94)


# Validation_datagenでもAugumentationをすれば、TTA的に若干の精度向上が見込まれる
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    dataframe = validation_df,
    #dataframe = val_data_gen(training_item_count, data_seed),
    x_col='image_id',
    y_col='label',
    directory='/Users/kushi/TechLife/kaggle_road/06_cassava/dataset/train_images/',
    target_size=target_size,
    shuffle=False,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed = 94)

print(tes_img.shape)

# Model-Setting

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Add, Activation, Input
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
#from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# 自作モデル_02_Vit
def MultiHead_SelfAttention(inputs, embed_dim, num_heads):
    projection_dim = embed_dim // num_heads
    batch_size = K.int_shape(inputs)[0]

    query = Dense(embed_dim)(inputs)
    key   = Dense(embed_dim)(inputs)
    value = Dense(embed_dim)(inputs)

    query = K.reshape(query, (batch_size, -1, num_heads, projection_dim))
    key   = K.reshape(key,   (batch_size, -1, num_heads, projection_dim))
    value = K.reshape(value, (batch_size, -1, num_heads, projection_dim))

    query = K.permute_dimensions(query, (0, 2, 1, 3))
    key   = K.permute_dimensions(key,   (0, 2, 1, 3))
    value = K.permute_dimensions(value, (0, 2, 1, 3))

    score = tf.matmul(query, key, transpose_b=True)
    score = score/K.sqrt(K.cast(projection_dim, 'float32'))
    weights = Activation('softmax')(score)

    attention = tf.matmul(weights, value)
    attention = K.permute_dimensions(attention, (0, 2, 1, 3))
    attention = K.reshape(attention, (batch_size, -1, embed_dim))
    output = Dense(embed_dim)(attention)
    return output

def TransformerBlock(inputs, embed_dim, num_heads, ff_dim):
    attn_output = MultiHead_SelfAttention(inputs, embed_dim, num_heads)
    attn_output = Dropout(0.1)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(Add()([inputs, attn_output]))
    ffn_output = Dense(ff_dim, activation="relu")(out1)
    ffn_output = Dense(embed_dim)(ffn_output)
    ffn_output = Dropout(0.1)(ffn_output)
    return LayerNormalization(epsilon=1e-6)(Add()([out1, ffn_output]))

class Add_Embedding_Layer(tf.keras.layers.Layer):
    def __init__(self, num_patches=64, d_model=64, batch_size=16):
        super(Add_Embedding_Layer, self).__init__()
        self.batch_size = batch_size
        self.patch_emb = self.add_weight(shape=[1, 1, d_model], dtype=tf.float32)
        self.pos_emb = self.add_weight(shape=[1, num_patches+1, d_model], dtype=tf.float32)

    def call(self, input):
        patch_emb = K.repeat_elements(self.patch_emb, self.batch_size, axis=0)
        pos_emb = K.repeat_elements(self.pos_emb, self.batch_size, axis=0)
        return K.concatenate([input, patch_emb], axis=1) + pos_emb



def make_ViT(img_size = IMAGE_SIZE, ch_size = CH_NUM, patch_size = PATCH_SIZE,
             batch_size = BATCH_SIZE, num_layers = 4, d_model = IMAGE_SIZE,
             num_heads = 4, mlp_dim = 128, num_classes = CLASS):

    num_patches = (img_size // patch_size) ** 2
    patch_dim = ch_size * patch_size ** 2

    #inputs = Input(shape=(32, 32, 3))
    inputs = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, CH_NUM))
    
    x = Rescaling(1./255)(inputs)
    x = tf.nn.space_to_depth(x, patch_size)
    x = K.reshape(x, (-1, num_patches, patch_dim))
    x = Dense(d_model)(x)

    x = Add_Embedding_Layer(num_patches, d_model, batch_size)(x)
    for _ in range(num_layers):
        x = TransformerBlock(x, d_model, num_heads, mlp_dim)

    x = Dense(mlp_dim, activation=tfa.activations.gelu)(x[:, 0])
    x = Dropout(0.1)(x)
    y = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=inputs, outputs=y)

model = make_ViT()
model.summary()

# 自作のモデル_01
model = models.Sequential()
model.add(layers.Conv2D(16, (3,3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, CH_NUM)))
model.add(layers.MaxPooling2D(2, 2))
# The second convolution
model.add(layers.Conv2D(32, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
# The third convolution
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
# The fourth convolution
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
# Flatten the results to feed into a dense layer
model.add(layers.Flatten())
# 128 neuron in the fully-connected layer
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(CLASS, activation="softmax"))

model.summary()

In [ ]:
#input = Input(shape=X_train[0].shape)
input = Input(shape=tes_img.shape)


#model_base = VGG16(include_top=False, weights="imagenet", input_tensor=input, classes=CLASS)
#model_base = ResNet50(include_top=False, weights="imagenet", input_tensor=input, input_shape=None, pooling=None, classes=CLASS)
#model_base = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input, input_shape=None, pooling=None, alpha=1.0, depth_multiplier=1, classes=CLASS)
#model_base = VGG19(include_top=False, weights='imagenet', input_tensor=input, input_shape=None, pooling=None, classes=CLASS)

# ここEfficientNet(weights='noisy-student'"imagenet"も試す)
model_base = EfficientNetB0(input_shape=input_shape, weights="imagenet", include_top=False)

#model = VGG16(include_top=False, weights="imagenet", input_shape=input)
#model_name = "VGG16-fine"
model_name = "efnB4-fine"
#model_name = "Resnet50-fine"

print("model取得完了")
model_base.summary()

In [ ]:
# 畳み込みベースのvgg16を凍結せずに再学習層を残す
print("凍結するけど、再学習層を残す！")
model_base.trainable = True

# このフラグがFalseの層は凍結される
set_flg = False

# どの層まで凍結するか・・・。。
#VGG16は、block5_conv1
#resnet50は、res5c_branch2c ,res5c_branch2a
#efficientNetB4は、top_convか、block7b_project_conv、block7b_se_reduceあたり？？
# block6a_expand_convが、6の層をa以降を解凍する⇒メモリエラーで無理
# block7a_expand_conv、⇒メモリエラーで無理

for layer in model_base.layers:
    if layer.name == "top_conv":  #"block5_conv1"というレイヤー以下はすべて trainable を True とする。
        set_flg = True
    if set_flg == True:
        layer.trainable = True
        print("set trainable True 凍結しない！")
    else:
        layer.trainable = False
        print("set trainable False 凍結！")

# モデルの確認
model_base.summary()

In [ ]:
# 全結合層の追加
print("全結合層の追加")
model = models.Sequential()
model.add(model_base)
#model.add(layers.Flatten(input_shape=(1,1,500)))
#model.add(layers.Dense(500, activation='relu'))
#model.add(layers.Flatten(input_shape=(1,1,CLASS)))
#model.add(layers.Dense(CLASS, activation='relu'))

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.5))
#model.add(layers.Dense(CLASS, activation='softmax'))

# 全結合層の各初期化、正則化の設定はきちんとする必要がある
model.add(layers.Dense(CLASS, activation='softmax', kernel_initializer='uniform', kernel_regularizer=K.regularizers.l2(0.)))


"""
# kerasの全結合層で利用可能な正規化
K.regularizers.l1(0.)
K.regularizers.l2(0.)
K.regularizers.l1_l2(l1=0.01, l2=0.01)
"""

print("FineTuneing!!")

# めも
conv_base = VGG16(weights = 'imagenet' , include_top = False , input_tensor=input, classes=CLASS)
conv_base.trainable = False # Freeze VGG16 base
model_2 = Sequential()
model_2.add(conv_base)
model_2.add(layers.Flatten())
model_2.add(layers.Dense(512 , activation = "relu"))
model_2.add(layers.Dense(units = 5 , activation = "softmax"))
model_2.summary()
model_2.compile(loss = LOSS, optimizer = opt, metrics = METRICS)

In [ ]:
model.summary()

In [ ]:
model.compile(loss = LOSS, optimizer = opt, metrics = METRICS)

# 学習処理

In [ ]:
es = EarlyStopping(monitor="accuracy", min_delta=0.000, patience=20, verbose=1, mode='auto')

In [ ]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = validation_generator.n//validation_generator.batch_size

In [ ]:
STEP_SIZE_VALID,STEP_SIZE_TRAIN

In [ ]:
training_df.shape, validation_df.shape

In [ ]:
print(tf.__version__)

In [ ]:
# GPU使えているよね？？
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
# メモリ制限するらしい
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# チェックポイントで定期的にモデルを保存する
# ファイル名に(`str.format`を使って)エポック数を埋め込む
from keras.callbacks import ModelCheckpoint

#checkpoint_path = "/Users/kushi/TechLife/kaggle_road/06_cassava/my_models/cp-{epoch:04d}.ckpt"
checkpoint_path = "/Users/kushi/TechLife/kaggle_road/06_cassava/my_models/cp-{epoch:04d}.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = ModelCheckpoint(filepath = checkpoint_path, 
                                monitor='val_loss', 
                                verbose=0, 
                                save_best_only=False, 
                                save_weights_only=False, 
                                mode='min', 
                                period=1)
hist = History()

T = time.time()
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=validation_generator,
                              validation_steps=STEP_SIZE_VALID,
                              epochs=EPOCHS,
                              verbose=1,
                              callbacks=[es, cp_callback, hist],
                              shuffle=True,
                              workers=20
                             )

print("学習の処理時間：", str(time.time() - T), " [sec]")

# ここから

# KFold(fit_generatorで似た感じに作って)、アンサンブルする

# モデル作成_複数作るよ！

In [ ]:
# KFold用
input = Input(shape=tes_img.shape)
T_model_in = time.time()
#model_base = VGG16(include_top=False, weights="imagenet", input_tensor=input, classes=CLASS)
#model_base = ResNet50(include_top=False, weights="imagenet", input_tensor=input, input_shape=None, pooling=None, classes=CLASS)
#model_base = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input, input_shape=None, pooling=None, alpha=1.0, depth_multiplier=1, classes=CLASS)
#model_base = VGG19(include_top=False, weights='imagenet', input_tensor=input, input_shape=None, pooling=None, classes=CLASS)
#model_base = VGG16(include_top=False, weights="imagenet", input_shape=input)

use_model = ResNet50(include_top=False, weights="imagenet", input_tensor=input, input_shape=None, pooling=None, classes=CLASS)
#use_model = EfficientNetB2(input_shape=input_shape, weights="imagenet", include_top=False)


# ここEfficientNet(weights='noisy-student'"imagenet"も試す)
#model_base_1 = EfficientNetB0(input_shape=input_shape, weights="imagenet", include_top=False)
model_base_1 = use_model
model_base_1.trainable = True
set_flg = False
for layer in model_base_1.layers:
    if layer.name == "top_conv":  #"block5_conv1"というレイヤー以下はすべて trainable を True とする。
        set_flg = True
    if set_flg == True:
        layer.trainable = True
        print("set trainable True 凍結しない！")
    else:
        layer.trainable = False
        #print("set trainable False 凍結！")
print("全結合層の追加")
model_1 = models.Sequential()
model_1.add(model_base_1)
model_1.add(layers.GlobalAveragePooling2D())
model_1.add(layers.Dropout(0.5))
model_1.add(layers.Dense(CLASS, activation='softmax', kernel_initializer='uniform', kernel_regularizer=K.regularizers.l2(0.)))
model_1.compile(loss = LOSS, optimizer = opt, metrics = METRICS)
print("FineTuneing!!")
print("モデルの取得時間：", str(time.time() - T_model_in), "[sec]")


#model_base_2 = EfficientNetB0(input_shape=input_shape, weights="imagenet", include_top=False)
model_base_2 = use_model
model_base_2.trainable = True
set_flg = False
for layer in model_base_2.layers:
    if layer.name == "top_conv":  #"block5_conv1"というレイヤー以下はすべて trainable を True とする。
        set_flg = True
    if set_flg == True:
        layer.trainable = True
        print("set trainable True 凍結しない！")
    else:
        layer.trainable = False
        #print("set trainable False 凍結！")
print("全結合層の追加")
model_2 = models.Sequential()
model_2.add(model_base_2)
model_2.add(layers.GlobalAveragePooling2D())
model_2.add(layers.Dropout(0.5))
model_2.add(layers.Dense(CLASS, activation='softmax', kernel_initializer='uniform', kernel_regularizer=K.regularizers.l2(0.)))
model_2.compile(loss = LOSS, optimizer = opt, metrics = METRICS)
print("FineTuneing!!")
print("モデルの取得時間：", str(time.time() - T_model_in), "[sec]")


#model_base_3 = EfficientNetB0(input_shape=input_shape, weights="imagenet", include_top=False)
model_base_3 = use_model
model_base_3.trainable = True
set_flg = False
for layer in model_base_3.layers:
    if layer.name == "top_conv":  #"block5_conv1"というレイヤー以下はすべて trainable を True とする。
        set_flg = True
    if set_flg == True:
        layer.trainable = True
        print("set trainable True 凍結しない！")
    else:
        layer.trainable = False
        #print("set trainable False 凍結！")
print("全結合層の追加")
model_3 = models.Sequential()
model_3.add(model_base_3)
model_3.add(layers.GlobalAveragePooling2D())
model_3.add(layers.Dropout(0.5))
model_3.add(layers.Dense(CLASS, activation='softmax', kernel_initializer='uniform', kernel_regularizer=K.regularizers.l2(0.)))
model_3.compile(loss = LOSS, optimizer = opt, metrics = METRICS)
print("FineTuneing!!")
print("モデルの取得時間：", str(time.time() - T_model_in), "[sec]")


#model_base_4 = EfficientNetB0(input_shape=input_shape, weights="imagenet", include_top=False)
model_base_4 = use_model
model_base_4.trainable = True
set_flg = False
for layer in model_base_4.layers:
    if layer.name == "top_conv":  #"block5_conv1"というレイヤー以下はすべて trainable を True とする。
        set_flg = True
    if set_flg == True:
        layer.trainable = True
        print("set trainable True 凍結しない！")
    else:
        layer.trainable = False
        #print("set trainable False 凍結！")
print("全結合層の追加")
model_4 = models.Sequential()
model_4.add(model_base_4)
model_4.add(layers.GlobalAveragePooling2D())
model_4.add(layers.Dropout(0.5))
model_4.add(layers.Dense(CLASS, activation='softmax', kernel_initializer='uniform', kernel_regularizer=K.regularizers.l2(0.)))
model_4.compile(loss = LOSS, optimizer = opt, metrics = METRICS)
print("FineTuneing!!")
print("モデルの取得時間：", str(time.time() - T_model_in), "[sec]")


#model_base_5 = EfficientNetB0(input_shape=input_shape, weights="imagenet", include_top=False)
model_base_5 = use_model
model_base_5.trainable = True
set_flg = False
for layer in model_base_5.layers:
    if layer.name == "top_conv":  #"block5_conv1"というレイヤー以下はすべて trainable を True とする。
        set_flg = True
    if set_flg == True:
        layer.trainable = True
        print("set trainable True 凍結しない！")
    else:
        layer.trainable = False
        #print("set trainable False 凍結！")  
print("全結合層の追加")
model_5 = models.Sequential()
model_5.add(model_base_5)
model_5.add(layers.GlobalAveragePooling2D())
model_5.add(layers.Dropout(0.5))
model_5.add(layers.Dense(CLASS, activation='softmax', kernel_initializer='uniform', kernel_regularizer=K.regularizers.l2(0.)))
model_5.compile(loss = LOSS, optimizer = opt, metrics = METRICS)
print("FineTuneing!!")

cv_models = [model_1, model_2, model_3, model_4, model_5]
cv_models[4].summary()

print("model取得完了")
print("モデルの取得時間：", str(time.time() - T_model_in), "[sec]")


In [ ]:
cv_models[4].layers[0].summary()

In [ ]:
print(use_model)

In [ ]:
# SKFold用のデータジェネレータ
#CV_NUM = 0
train_datagen = ImageDataGenerator(
    rotation_range=30,
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip = True,
    height_shift_range = 0.15,
    width_shift_range = 0.15
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe = training_df,
    #dataframe = train_data_gen(training_item_count, data_seed),
    x_col='image_id',
    y_col='label',
    directory='/Users/kushi/TechLife/kaggle_road/06_cassava/dataset/train_images/',
    target_size=target_size,
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='categorical',
    seed = 94)


# Validation_datagenでもAugumentationをすれば、TTA的に若干の精度向上が見込まれる
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    dataframe = validation_df,
    #dataframe = val_data_gen(training_item_count, data_seed),
    x_col='image_id',
    y_col='label',
    directory='/Users/kushi/TechLife/kaggle_road/06_cassava/dataset/train_images/',
    target_size=target_size,
    shuffle=False,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed = 94)

In [ ]:
# 学習前準備
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = validation_generator.n//validation_generator.batch_size

In [ ]:
CV_NUM = 0
# コールバック全般設定(出来れば学習中に変化させたい)
# 学習率のスケジューラ(適当に作っているので、、、
def lr_schedul(epoch):
    x = 0.01
    if epoch >= 1:
        x = 0.005
    elif epoch >= 3:
        x = 0.0001
    elif epoch >= 6:
        x = 0.0005
    
    print("学習率は：『 ", str(x), " 』")
    #print("  GPU使用状況の表示  ")
    #pprint.pprint(get_gpu_info())
    return x
lr_decay = LearningRateScheduler(lr_schedul, verbose=1)

# 学習率の上昇
reduce_lr = ReduceLROnPlateau(
                        monitor='val_loss',
                        factor=0.2,
                        patience=2,
                        min_lr=0.00001
                )

# 早期停止条件
es = EarlyStopping(monitor="accuracy", min_delta=0.001, patience=2, verbose=1, mode='auto')

# チェックポイントで定期的にモデルを保存する
# ファイル名に(`str.format`を使って)エポック数を埋め込む
#checkpoint_path = "/Users/kushi/TechLife/kaggle_road/06_cassava/my_models/cp-{epoch:04d}.ckpt"
#checkpoint_path = "/Users/kushi/TechLife/kaggle_road/06_cassava/my_models/model_" + str(CV_NUM) + "_cp-{epoch:04d}.h5"
#checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = ModelCheckpoint(filepath = "/Users/kushi/TechLife/kaggle_road/06_cassava/my_models/model_" + str(CV_NUM) + "_cp-{epoch:04d}.h5", 
                                monitor='val_accuracy', 
                                verbose=0, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='min', 
                                period=1)
hist = History()

In [ ]:
# 評価用画像の準備
tes00 = get_img(test_imgs[0])
tes_im = tes00[np.newaxis,:,:,:]

In [ ]:
# KFold的にfit_generatorを使う
n_splits = 5
T0 = time.time()

for CV_NUM in range(n_splits):
    T = time.time()
    print("")
    print("cv_num：", str(CV_NUM))
    training_df = train_data_gen(training_item_count, data_seed)
    validation_df = val_data_gen(training_item_count, data_seed)
    
    history = cv_models[CV_NUM].fit_generator(generator=train_generator,
                                              steps_per_epoch=STEP_SIZE_TRAIN,
                                              validation_data=validation_generator,
                                              validation_steps=STEP_SIZE_VALID,
                                              epochs=EPOCHS,
                                              verbose=1,
                                              callbacks=[reduce_lr, es, cp_callback, hist],
                                              shuffle=True,
                                              workers=10
                                             )
    
    # 評価工程
    pred = cv_models[CV_NUM].predict(tes_im)
    result = np.argmax(pred)
    print("●〇●〇●モデル No：", str(CV_NUM))
    print(result)
    print(pred)
    
    # 学習過程のhistoryを保存
    hist_df = pd.DataFrame(hist.history)
    hist_df.to_csv("/Users/kushi/TechLife/kaggle_road/06_cassava/my_models/history_model_" + str(CV_NUM) + ".csv")
    # 学習後のモデルを丸ごと保存！
    cv_models[CV_NUM].save("/Users/kushi/TechLife/kaggle_road/06_cassava/my_models/model_" + str(CV_NUM) + ".h5", include_optimizer=False)
    print("SKFoldの処理時間：", str(time.time() - T), " [sec]")

print("学習の処理時間：", str(time.time() - T0), " [sec]")

In [ ]:
# 評価用画像について
test_imgs[0]

In [ ]:
tes00 = get_img(test_imgs[0])
plt.imshow(tes00)
print(tes00.shape)

In [ ]:
tes01 = tes00[np.newaxis,:,:,:]
print(tes01.shape)

In [ ]:
pred = model.predict(tes01)

In [ ]:
result = np.argmax(pred)
print(result)
print(pred)

In [ ]:
# 学習後のモデルを丸ごと保存！
model.save("/Users/kushi/TechLife/kaggle_road/06_cassava/my_models/model.h5", include_optimizer=False)

In [ ]:
# histは、callbackで設定してる変数
#hist_df = pd.DataFrame(history.history)
hist_df = pd.DataFrame(hist.history)
hist_df.to_csv("/Users/kushi/TechLife/kaggle_road/06_cassava/my_models/history.csv")

In [ ]:
hist_df.head(20)

In [ ]:
# Acc画像の保存
plt.figure()
hist_df[['accuracy', 'val_accuracy']].plot()
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.savefig(os.path.join("/Users/kushi/TechLife/kaggle_road/06_cassava/my_models/", "acc.png"))
plt.close()

In [ ]:
# loss画像の保存
plt.figure()
hist_df[['loss', 'val_loss']].plot()
plt.ylabel('loss')
plt.xlabel('epoch')
plt.savefig(os.path.join("/Users/kushi/TechLife/kaggle_road/06_cassava/my_models/", "loss.png"))
plt.close()

In [ ]:
# train_labelのデータフレームを表示、なぜか先頭の1枚目が無かったので削除する
print(train_label.shape)

train_label2 = train_label[1:]
train_label2.shape
train_label2 =train_label2.reset_index()

In [ ]:
train_label2.head()

In [ ]:
10%10

In [ ]:
# 混合行列
# 全データやると、だいたい560secかかるよ！
T = time.time()
pred_list = []
i=0
for i in range(len(imgs)//1):
    im = get_img(imgs[i])
    #print(imgs[i])
    im = im[np.newaxis, :, :, :]
    pred = model.predict(im)
    pred = np.argmax(pred)
    #print(pred)
    
    if i == 0:
        pred_list = pred
    elif i != 0:
        pred_list = np.append(pred_list, pred)
    
    if i % 100 == 0:
        print("かうんと", str(i))
    #print(pred_list)
print("処理時間：", str(time.time() - T), " [sec]")    
#print(confusion_matrix(y_true_multi, y_pred_multi))

In [ ]:
pred_list.shape

In [ ]:
pred_list[:100]

In [ ]:
pred_list_df = pd.DataFrame(pred_list)
#train_label2 = pd.DataFrame(train_label2)
pred_list_df.columns = ["pred"]
pred_list_df.astype(int)
print(pred_list_df.shape)
print(train_label2.shape)

In [ ]:
train_label2.head()

In [ ]:
pred_list_df.head()

In [ ]:
pred_list_df["pred"].value_counts()

In [ ]:
train_label2["label"].value_counts()

In [ ]:
print(type(pred_list_df))
print(type(train_label2))
print("pred_list_df", 
      pred_list_df.dtypes)

print("train_label2",
      train_label2.dtypes)

In [ ]:
result_df = pd.concat([train_label2, pred_list_df], axis=1)
print(result_df.dtypes)
print(result_df.shape)

In [ ]:
result_df.head(30)

In [ ]:
result_df.to_csv("/Users/kushi/TechLife/kaggle_road/06_cassava/my_models/Add_Result.csv")

In [ ]:
# 混合行列
cm = confusion_matrix(result_df["label"], result_df["pred"])
print(cm)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report


In [ ]:
print(accuracy_score(result_df["label"], result_df["pred"]))
print(precision_score(result_df["label"], result_df["pred"], average="macro"))
print(recall_score(result_df["label"], result_df["pred"], average='macro'))
print(f1_score(result_df["label"], result_df["pred"], average='macro'))

In [ ]:
print(classification_report(result_df["label"], result_df["pred"]))

In [ ]:
end

In [ ]:
# 学習用
#history = model.fit(train_generator,
##                  epochs = EPOCHS, 
#                 validation_data=validation_generator)

In [ ]:
x_test.shape

In [ ]:
#decay_steps = int(round(len(training_df)/batch_size))*epochs
#cosine_decay = CosineDecay(initial_learning_rate=1e-4, decay_steps=decay_steps, alpha=0.2)

#callbacks = [ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

#model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(cosine_decay), metrics=["accuracy"])

In [ ]:
pred = model.predict(x_test, batch_size=batch_size)

In [ ]:
for i in range(20):
    print(np.argmax(pred[i]), "：確率は　", np.max(pred[i]), "=======正解⇒", np.argmax(y_test[i]))

In [ ]:
cm = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(pred, axis=1))
print(cm)

In [ ]:
# 学習中の損失係数とAccuracyを可視化
fig, ax = plt.subplots(2,1)

ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)